In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
method = "phm"
dataset = "test_data"
folder_name = "no_TADs_no_AB_compartments"
filtering_type = "stringent_filtering_by_length_and_gc_content"
# "stringent_filtering_by_length_and_gc_content_and_k27ac" <- Normally would use this but there were no CMs passing the filtering.
core_path = os.path.join(
    "/data/pushkare/Chromatin_modules/3.simulate_CMs/output",
    folder_name,
    "QC",
    filtering_type,
    method,
)

if not os.path.exists(os.path.join(core_path, "peaks")):
    os.makedirs(os.path.join(core_path, "peaks"))

output_path = os.path.join(core_path, "peaks")

In [3]:
for ref_sim in ["ref", "sim"]:
    content_df = pd.read_csv(
        os.path.join(
            core_path,
            "_".join(
                [
                    dataset,
                    ref_sim,
                    method,
                    "length_GC_content_filtering.content.txt",
                ]
            ),
        ),
        sep="\t",
        header=None,
        names=["cm_id", "n_peaks", "peaks"],
    )
    if not "chr" in content_df.iloc[0, 2].split(",")[0]:
        for i in np.arange(1, 23):
            content_df.iloc[:, 2] = content_df.iloc[:, 2].str.replace(
                ":" + str(i) + ":", ":chr" + str(i) + ":"
            )
    cm_peak_dict = dict(
        zip(
            content_df.loc[:, "cm_id"].to_list(),
            content_df.loc[:, "peaks"].str.split(",").to_list(),
        )
    )
    peak_info_list = []
    for cm_id, cm_peaks in cm_peak_dict.items():
        for cm_peak in cm_peaks:
            peak_info_list.append(cm_peak.split(":") + [cm_peak, cm_id])
    peak_df = pd.DataFrame(
        peak_info_list,
        columns=["mark", "chromosome", "start", "end", "pid", "cm_id"],
    )
    peak_df.drop("mark", axis=1, inplace=True)
    peak_df.loc[:, "strand"] = "+"
    peak_df.to_csv(
        os.path.join(output_path, "_".join([dataset, method, ref_sim, "peaks.bed"])),
        sep="\t",
        index=False,
        header=False,
    )